In [1]:
# Import the required libraries
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
# Load the data into pandas dataframes. The data has to be manually saved to a folder called 'data'.
# Note: the data is quite large, so this may take a while (~40 seconds) if the data is read as a csv. To speed up further file reading, it is converted to pickle format.
if (os.path.exists(os.path.join('data', 'training_set_VU_DM.pickle'))
    & os.path.exists((os.path.join('data', 'test_set_VU_DM.pickle')))):
    train_df = pd.read_pickle(os.path.join('data', 'training_set_VU_DM.pickle'))
    test_df = pd.read_pickle(os.path.join('data', 'test_set_VU_DM.pickle'))
else:
    train_df = pd.read_csv(os.path.join('data', 'training_set_VU_DM.csv'))
    test_df = pd.read_csv(os.path.join('data', 'test_set_VU_DM.csv'))
    train_df.to_pickle(os.path.join('data', 'training_set_VU_DM.pickle'))
    test_df.to_pickle(os.path.join('data', 'test_set_VU_DM.pickle'))

In [3]:
print(f"Period of data collection: {pd.to_datetime(train_df['date_time']).min().strftime('%Y/%m/%d')} - {pd.to_datetime(train_df['date_time']).max().strftime('%Y/%m/%d')}")
print(f"Train data contains {train_df.shape[0]:,} rows and {train_df.shape[1]} columns")
print(f"Test data contains {test_df.shape[0]:,} rows and {test_df.shape[1]} columns")
print()
print(f"Train data:")
print(f"Number of unique search IDs: {len(train_df['srch_id'].unique()):,}")
print(f"Number of unique property IDs: {len(train_df['prop_id'].unique()):,}")
print(f"Number of clicks per search: avg. {train_df['click_bool'].sum() / len(train_df['srch_id'].unique()):.2f}, std. {train_df['click_bool'].std():.2f}")
print(f"Number of bookings per search: avg. {train_df['booking_bool'].sum() / len(train_df['srch_id'].unique()):.2f}, std. {train_df['booking_bool'].std():.2f}")
print()
print(f"Test data:")
print(f"Number of unique search IDs: {len(test_df['srch_id'].unique()):,}")
print(f"Number of unique property IDs: {len(test_df['prop_id'].unique()):,}")

Period of data collection: 2012/11/01 - 2013/06/30
Train data contains 4,958,347 rows and 54 columns
Test data contains 4,959,183 rows and 50 columns

Train data:
Number of unique search IDs: 199,795
Number of unique property IDs: 129,113
Number of clicks per search: avg. 1.11, std. 0.21
Number of bookings per search: avg. 0.69, std. 0.16

Test data:
Number of unique search IDs: 199,549
Number of unique property IDs: 129,438


### Data Columns

| Column Name                 | Data Type | Description                                                                                                                                                                                                       |
|-----------------------------|-----------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| position                    | Integer   | Hotel position on Expedia's search results page. This is only provided for the training data, but not the test data.                                                                                              |
| gross_booking_usd           | Float     | Total value of the transaction. This can differ from the price_usd due to taxes, fees, conventions on multiple day bookings and purchase of a room type other than the one shown in the search                    |
| click_bool                  | Boolean   | 1 if the user clicked on the property, 0 if not.                                                                                              |
| booking_bool                | Boolean   | 1 if the user booked the property, 0 if not.                    |
|                             |           ||
| srch_id                     | Integer   | The ID of the search                                                                                                                                                                                              |
| date_time                   | Date/time | Date and time of the search                                                                                                                                                                                       |
| site_id                     | Integer   | ID of the Expedia point of sale (i.e. Expedia.com, Expedia.co.uk, Expedia.co.jp, ..)                                                                                                                              |
| visitor_location_country_id | Integer   | The ID of the country the customer is located                                                                                                                                                                     |
| visitor_hist_starrating     | Float     | The mean star rating of hotels the customer has previously purchased; null signifies there is no purchase history on the customer                                                                                 |
| visitor_hist_adr_usd        | Float     | The mean price per night (in US$) of the hotels the customer has previously purchased; null signifies there is no purchase history on the customer                                                                |
| prop_country_id             | Integer   | The ID of the country the hotel is located in                                                                                                                                                                     |
| prop_id                     | Integer   | The ID of the hotel                                                                                                                                                                                               |
| prop_starrating             | Integer   | The star rating of the hotel, from 1 to 5, in increments of 1. A 0 indicates the property has no stars, the star rating is not known or cannot be publicized.                                                     |
| prop_review_score           | Float     | The mean customer review score for the hotel on a scale out of 5, rounded to 0.5 increments. A 0 means there have been no reviews, null that the information is not available.                                    |
| prop_brand_bool             | Integer   | +1 if the hotel is part of a major hotel chain; 0 if it is an independent hotel                                                                                                                                   |
| prop_location_score1        | Float     | A (first) score outlining the desirability of a hotel’s location                                                                                                                                                  |
| prop_location_score2        | Float     | A (second) score outlining the desirability of the hotel’s location                                                                                                                                               |
| prop_log_historical_price   | Float     | The logarithm of the mean price of the hotel over the last trading period. A 0 will occur if the hotel was not sold in that period.                                                                               |
| price_usd                   | Float     | Displayed price of the hotel for the given search. Note that different countries have different conventions regarding displaying taxes and fees and the value may be per night or for the whole stay              |
| promotion_flag              | Integer   | +1 if the hotel had a sale price promotion specifically displayed                                                                                                                                                 |
| srch_destination_id         | Integer   | ID of the destination where the hotel search was performed                                                                                                                                                        |
| srch_length_of_stay         | Integer   | Number of nights stay that was searched                                                                                                                                                                           |
| srch_booking_window         | Integer   | Number of days in the future the hotel stay started from the search date                                                                                                                                          |
| srch_adults_count           | Integer   | The number of adults specified in the hotel room                                                                                                                                                                  |
| srch_children_count         | Integer   | The number of (extra occupancy) children specified in the hotel room                                                                                                                                              |
| srch_room_count             | Integer   | Number of hotel rooms specified in the search                                                                                                                                                                     |
| srch_saturday_night_bool    | Boolean   | +1 if the stay includes a Saturday night, starts from Thursday with a length of stay is less than or equal to 4 nights (i.e. weekend); otherwise 0                                                                |
| srch_query_affinity_score   | Float     | The log of the probability a hotel will be clicked on in Internet searches (hence the values are negative)  A null signifies there are no data (i.e. hotel did not register in any searches)                      |
| orig_destination_distance   | Float     | Physical distance between the hotel and the customer at the time of search. A null means the distance could not be calculated.                                                                                    |
| random_bool                 | Boolean   | +1 when the displayed sort was random, 0 when the normal sort order was displayed                                                                                                                                 |
| comp*x*_rate                | Integer   | '*x*' denotes the competitor number. +1 if Expedia has a lower price than competitor 1 for the hotel; 0 if the same; -1 if Expedia’s price is higher than competitor 1; null signifies there is no competitive data |
| comp*x*_inv                 | Integer   | '*x*' denotes the competitor number. +1 if competitor 1 does not have availability in the hotel; 0 if both Expedia and competitor 1 have availability; null signifies there is no competitive data                  |
| comp*x*_rate_percent_diff   | Float     | '*x*' denotes the competitor number. The absolute percentage difference (if one exists) between Expedia and competitor 1’s price (Expedia’s price the denominator); null signifies there is no competitive data      |


# Data Transformation

In [4]:
def create_features(df):
    """
    Create the following new features: has_starrating, has_review_score, traveling_abroad, srch_prop_country_match, month, and day_of_week
    """
    # has_starrating: boolean whether prop_starrating is 0 or null
    df["has_starrating"] = df["prop_starrating"].isnull()
    df["has_starrating"] = df["has_starrating"].astype(int)
    df.loc[df["prop_starrating"] == 0, "has_starrating"] = 1

    # has_review_score: boolean whether prop_review_score is 0 or null
    df["has_review_score"] = df["prop_review_score"].isnull()
    df["has_review_score"] = df["has_review_score"].astype(int)
    df.loc[df["prop_review_score"] == 0, "has_review_score"] = 1

    # traveling_abroad: boolean whether visitor_location_country_id != prop_country_id
    df["traveling_abroad"] = df["visitor_location_country_id"] != df["prop_country_id"]
    df["traveling_abroad"] = df["traveling_abroad"].astype(int)

    # srch_prop_country_match: boolean whether srch_destination_id == prop_country_id
    df["srch_prop_country_match"] = df["srch_destination_id"] == df["prop_country_id"]
    df["srch_prop_country_match"] = df["srch_prop_country_match"].astype(int)

    # month: month of the search, one-hot encoded
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["month"] = df["date_time"].dt.month
    df["month"] = df["month"].map({1: "jan", 2: "feb", 3: "mar", 4: "apr", 5: "may", 6: "jun", 7: "jul", 8: "aug", 9: "sep", 10: "oct", 11: "nov", 12: "dec"})
    df = pd.get_dummies(df, columns=["month"], dtype=int)
    for col in ["month_jan", "month_feb", "month_mar", "month_apr", "month_may", "month_jun", "month_jul", "month_aug", "month_sep", "month_oct", "month_nov", "month_dec"]:
        if col not in df.columns:
            df[col] = 0

    # day_of_week: day of the week of the search
    df["day_of_week"] = df["date_time"].dt.dayofweek
    df["day_of_week"] = df["day_of_week"].map({0: "mon", 1: "tue", 2: "wed", 3: "thu", 4: "fri", 5: "sat", 6: "sun"})
    df = pd.get_dummies(df, columns=["day_of_week"], dtype=int)
    for col in ["day_of_week_mon", "day_of_week_tue", "day_of_week_wed", "day_of_week_thu", "day_of_week_fri", "day_of_week_sat", "day_of_week_sun"]:
        if col not in df.columns:
            df[col] = 0

    return df

In [5]:
def drop_nan_columns(df):
    """
    Train data shows that "visitor_hist_starrating", "visitor_hist_adr_usd", "srch_query_affinity_score", and "compx_rate_percent_diff" have >90% NaN values. These values cannot be imputed accurately, so we drop these columns.
    """
    cols = ["visitor_hist_starrating", "visitor_hist_adr_usd", "srch_query_affinity_score"] + [f"comp{i}_rate_percent_diff" for i in range(1, 9)]
    df.drop(columns=cols, inplace=True)
    return df

In [6]:
def impute_missing_values(df):
    """
    Impute missing values for the following columns: "prop_starrating", "prop_review_score", "compx_rate", and "compx_inv".

    For "prop_starrating" and "prop_review_score", we replace 0 values with NaN and then impute the NaN values with the mean per srch_id. Remaining NaN values are then filled with 0.
    For "compx_rate" and "compx_inv", we assume that missing data means that Expedia has the same price and equal availability as its competitors. We therefore impute the NaN values with 0.
    """
    # Replace 0 values with NaN
    df["prop_starrating"] = df["prop_starrating"].replace(0, np.nan)
    df["prop_review_score"] = df["prop_review_score"].replace(0, np.nan)
    # Impute NaN values with mean per srch_id
    df["prop_starrating"] = df.groupby("srch_id")["prop_starrating"].transform(lambda x: x.fillna(x.mean()))
    df["prop_review_score"] = df.groupby("srch_id")["prop_review_score"].transform(lambda x: x.fillna(x.mean()))
    # Fill remaining NaN values with 0
    df["prop_starrating"] = df["prop_starrating"].fillna(0)
    df["prop_review_score"] = df["prop_review_score"].fillna(0)

    # Impute NaN values with 0
    for i in range(1, 9):
        df[f"comp{i}_rate"] = df[f"comp{i}_rate"].fillna(0)
        df[f"comp{i}_inv"] = df[f"comp{i}_inv"].fillna(0)

    return df

In [7]:
def compute_aggregated_values(df):
    """
    Compute the mean, median and standard deviation for the following columns:
    "visitor_hist_starrating", "visitor_hist_adr_usd", "prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"
    """
    numerical_cols = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"]
    # srch_length_of_stay, srch_booking_window, srch_adults_count, srch_children_count, and srch_room_count are also numerical variables, but it has no use aggregating these values over prop_id.

    agg_df = df.groupby("prop_id").agg({col: ["mean", "std", "median"] for col in numerical_cols})
    agg_df.columns = ["_".join(col) for col in agg_df.columns]
    for col in agg_df.columns:
        df[col] = df["prop_id"].map(agg_df[col])
    return df

In [8]:
def compute_relative_values(df):
    """
    Subtract the mean per srch_id from the following columns:
    "prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"

    This is done so that the model can learn the relative values of these columns per srch_id.
    """
    cols = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_log_historical_price", "price_usd"]
    grouper = df.groupby('srch_id')
    for col in cols:
        df[col] = df[col] - grouper[col].transform('mean')
    return df

In [9]:
def drop_columns(df):
    """
    Train data shows that for "orig_destination_distance" over 75% of the data with a calculated value lower than 0.95 of the largest distance was lower than 130, meaning that the distance per srch_id is roughly the same. We assume therefore that this is not a deciding factor for a customer in their booking process and drop this column.

    Features were created from "date_time" and the column will not be used anymore, so we drop this column as well.

    Columns containing IDs ("site_id", "visitor_location_country_id", "prop_country_id", "prop_id", and "srch_destination_id") are not used in the model, so we drop these columns as well. "srch_id" and "prop_id" will remain in the columns for now for later use.

    If the supplied dataframe is the training dataframe, drop the unused target columns as well.

    # TODO:
    I don't really know what to do with "prop_location_score2" yet, so I'll drop it for now.
    """
    df.drop(columns=["orig_destination_distance"], inplace=True)
    df.drop(columns=["date_time"], inplace=True)
    df.drop(columns=["site_id", "visitor_location_country_id", "prop_country_id", "srch_destination_id"], inplace=True)
    df.drop(columns=["prop_location_score2"], inplace=True)
    for col in ["position", "gross_bookings_usd"]:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
    return df

In [10]:
def process_data(train_df: pd.DataFrame, test_df: pd.DataFrame, target_value: str = "booking_bool"):
    """
    Process the dataframes for training and testing the model.
    :param train_df: The training dataframe
    :param test_df: The test dataframe
    :param target_value: The target value to use for training the model. Either "booking_bool" or "both".
    :return: The processed dataframes
    """
    # Create features
    train_df = create_features(train_df)
    test_df = create_features(test_df)

    # Drop columns with NaN values
    train_df = drop_nan_columns(train_df)
    test_df = drop_nan_columns(test_df)

    # Impute missing values
    train_df = impute_missing_values(train_df)
    test_df = impute_missing_values(test_df)

    # Compute aggregated values
    train_df = compute_aggregated_values(train_df)
    test_df = compute_aggregated_values(test_df)

    # Compute relative values
    train_df = compute_relative_values(train_df)
    test_df = compute_relative_values(test_df)

    # Drop columns
    train_df = drop_columns(train_df)
    test_df = drop_columns(test_df)

    # Split train data into features and target
    if target_value == "booking_bool":
        train_df["target"] = train_df["booking_bool"]
    elif target_value == "both":
        train_df["target"] = 5*train_df["booking_bool"] + train_df["click_bool"]
    else:
        raise ValueError("target_value must be either 'booking_bool' or 'both'")

    train_df.drop(columns=["booking_bool", "click_bool"], inplace=True)

    return train_df, test_df

### DataLoaders

In [11]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.data = df

    def __getitem__(self, idx):
        row = self.data[idx]
        return row["srch_id"], torch.tensor(row["sequence"].astype(np.float64)), torch.tensor(row["target"].astype(np.int32))

    def __len__(self):
        return len(self.data)

In [12]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.data = df

    def __getitem__(self, idx):
        row = self.data[idx]
        return row["srch_id"], torch.tensor(row["sequence"].astype(np.float64))

    def __len__(self):
        return len(self.data)

In [13]:
def generate_sequences(df: pd.DataFrame, kind: str = "train") -> dict:
    """
    Generate sequences from the dataframe.
    :param df: The dataframe to take the sequences from.
    :param kind: The kind of dataframe. Either "train" or "test".
    :return: Dictionary containing the sequences and targets for the training data.
    """
    sequences = []
    # Very slow. Feel free to improve if you know how, I'm at a blank.
    if kind == "train":
        for srch_id, group in df.groupby("srch_id"):
            prop_ids = group["prop_id"].values
            sequence = group.drop(columns=["srch_id", "target"]).values
            target = group["target"].values
            sequences.append({"srch_id": srch_id, "prop_id": prop_ids, "sequence": sequence, "target": target})
    elif kind == "test":
        for srch_id, group in df.groupby("srch_id"):
            prop_ids = group["prop_id"].values
            sequence = group.drop(columns=["srch_id"]).values
            sequences.append({"srch_id": srch_id, "prop_id": prop_ids, "sequence": sequence})
    else:
        raise ValueError("kind must be either 'train' or 'test'")
    return sequences

In [14]:
def create_dataloaders(train_df: pd.DataFrame, test_df: pd.DataFrame, target_value: str = "booking_bool", batch_size: int=1, train_validation_split: float=0.8, shuffle: bool=True, force_data_processing: bool = False) -> tuple[DataLoader, DataLoader, DataLoader]:
    """
    Create dataloaders for the training, validation and test set.
    :param train_df: The train dataframe.
    :param test_df: The test dataframe.
    :param target_value: The target value to use for training the model. Either "booking_bool" or "both".
    :param batch_size: How many samples per batch to load.
    :param train_validation_split: The percentage of the dataset to use for training.
    :param shuffle: Set to True to have the data reshuffled at every epoch.
    :param force_data_processing: Set to True to force the data to be processed again.
    :return: Tuple containing the training, validation and test dataloaders.
    """
    if os.path.exists(os.path.join('data', 'final_train_set.pickle')) \
            & os.path.exists(os.path.join('data', 'final_test_set.pickle')) \
            & (not force_data_processing):
        train_df = pd.read_pickle(os.path.join('data', f'final_train_set_{target_value}.pickle'))
        test_df = pd.read_pickle(os.path.join('data', f'final_test_set_{target_value}.pickle'))
    else:
        train_df, test_df = process_data(train_df, test_df, target_value=target_value)
        train_df.to_pickle(os.path.join('data', f'final_train_set_{target_value}.pickle'))
        test_df.to_pickle(os.path.join('data', f'final_test_set_{target_value}.pickle'))

    train_sequences = generate_sequences(train_df, kind="train")
    train_dataset = TrainDataset(train_sequences)
    train_size = int(train_validation_split * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    test_sequences = generate_sequences(test_df, kind="test")
    test_dataset = TestDataset(test_sequences)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_dataloader, val_dataloader, test_dataloader

**Models**

In [1]:
# Relevant imports
import torch
import torch.nn as nn

In [ ]:
# Load all data
train_loader, dev_loader, test_loader = create_dataloaders(train_df, test_df, target_value="booking_bool", batch_size=1, train_validation_split=0.8, shuffle=True, force_data_processing=False)
# train_loader = ...
# dev_loader = ...
# test_loader = ...

In [ ]:
# Calculate NDCG@k score from file

def NDCG_from_file(fn, data, k):
    """
    Function to calculate the NDCG@k score from a txt file in the format specified in the assignment (two 
    columns, the left one containing the search IDs and the right one containing the PropertyIDs, sorted by
    relevance)
    """
    query_to_properties = {}
    
    # Fill the query-to-properties dictionary
    with open(fn) as f:
        lines = f.readlines()[1:] # Remove the header
        for line in lines:
            query_id, prop_id = line.split(",")
            query_id, prop_id = int(query_id), int(prop_id)
            if query_id not in query_to_properties.keys():
                query_to_properties[query_id] = [prop_id]
            else:
                query_to_properties[query_id].append(prop_id)

    NDCG_score = 0
    for query_id, prop_ids in query_to_properties.items():
        
        # TODO: change this to match the correct query
        data_query = data[query_id]

        # Only select the first k properties
        prop_selection = prop_ids[:k]

        # TODO: Change "scores" to match the name of the column which keeps the scores (5 for booking, 1 for clicking)
        true_scores = data.loc(data_query["prop_id"] == prop_selection, "scores")
    
        # Calculate DCG@k (Discounted Cumulative Gain at k)
        dcg = np.sum(true_scores / np.log2(np.arange(2, k+2)))
        
        # Sort the true scores in descending order
        true_sorted_indices = np.argsort(data_query["scores"])[::-1]
        true_sorted_scores = np.array(data_query["scores"])[true_sorted_indices]
        
        # Calculate ideal DCG@k
        ideal_dcg = np.sum(true_sorted_scores[:k] / np.log2(np.arange(2, k+2)))
        
        # Calculate NDCG@k
        NDCG_score += dcg / ideal_dcg if ideal_dcg > 0 else 0.0

    return NDCG_score

In [ ]:
# Simple Neural model
class RecommenderNet(nn.Module):
    """
    Neural prediction model for predicting the probability that a user will buy 
    a room at a certain hotel.
    """
    def __init__(self, num_features, hidden_size=64):
        super().__init__()
        self.embedding = nn.Linear(num_features, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 2*hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(2*hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, features):
        x = self.embedding(features)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        
        return x

_Training and testing code_

In [ ]:
import torch.optim as optim
from tqdm import tqdm

def train(model, criterion, num_epochs, train_loader, val_loader):
    train_loss = []
    val_loss = []    
    optimizer = optim.Adam(model.parameters())

    for _ in tqdm(range(num_epochs)):

        # Training code
        running_train_loss = 0
        for features, targets in train_loader:
            optimizer.zero_grad()
            
            outputs = model(features)
            loss = criterion(outputs, targets.unsqueeze(1))
            loss.backward()
            running_train_loss += loss.item()
            
            optimizer.step()
        train_loss.append(running_train_loss)

        # Validation loop
        running_val_loss = 0
        for features, targets in val_loader:
            with torch.no_grad:
                outputs = model(features)
                loss = criterion(outputs, targets.unsqueeze(1))
                running_val_loss += loss.item()

            val_loss.append(running_val_loss)

    return train_loss, val_loss


def test_and_write(fn, model, loader):
    with open(fn, "w") as f:
        f.write("SearchId, PropertyId")
        for elem in loader:

            # Check if there are two elements in the dataloader (a feature vector and a label).
            # If so, we have the validation set and need to extract the features accordingly
            if len(elem) == 2:
                features = elem[0]
            else:
                features = elem 

            # This assumes that features is batched and shaped (num_properties x num_features)
            search_id = features[:, 0] #TODO extract search ID from feature vector correctly

            model.eval()
            with torch.no_grad:
                # Don't take into account the search ID (first feature)
                outputs = model(features[:, 1:])
                probabilities = torch.sigmoid(outputs.squeeze())

                sorted_indices = probabilities.argsort(descending=True)    
            
            property_ids = features[sorted_indices][1] #TODO extract property ID from features
                
            for search, prop in zip(search_id, property_ids):
                f.writeln(f"{search}, {prop}")

In [ ]:
# Train simple NN model
num_epochs = 100 #TODO
num_features = 0 #TODO

simple_nn_model = RecommenderNet(num_features)
criterion = nn.BCEWithLogitsLoss()

nn_train_loss, nn_val_loss = train(simple_nn_model, criterion, num_epochs, train_loader, dev_loader)

# Test and save results of simple NN model
# For validation set
test_and_write("simple_nn_val.txt", simple_nn_model, dev_loader)
# And for testing set
test_and_write("simple_nn_test.txt", simple_nn_model, test_loader)

In [ ]:
# Calculate NDCG score for validation set
nn_score = NDCG_from_file("simple_nn_val.txt", train_df, 5)
print(f"NDGC score for the simple MLP (on validation set): {score}")